In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from statsmodels.tsa.ar_model import AutoReg
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
pxlst=['a.px', 'b.px', 'c.px', 'd.px']

In [3]:
def get_df_return(df):
    data={}
    for col in df.columns:
        lst = []
        now = df[col].values
        for i in range(1,len(now)):
            lst.append( np.log(now[i]/now[i-1]) )
        data[col]=lst
    df_res = pd.DataFrame(data)
    return df_res

In [4]:
df_mid = pd.read_csv('data/traindata.csv', index_col=[0])[pxlst]

In [9]:
df_mid.corr()

,a.px,b.px,c.px,d.px
a.px,1.000000,0.001853,0.000384,0.000490
b.px,0.001853,1.000000,-0.002087,-0.001787
c.px,0.000384,-0.002087,1.000000,0.151998
d.px,0.000490,-0.001787,0.151998,1.000000


In [27]:
df_mid.mean().values

array([ 99.99990317,  99.99837184, 100.97479159, 101.47371969])

In [24]:
df_mid.std().values

array([0.73413634, 0.73520834, 0.6112516 , 0.88995665])

In [6]:
df_mid_diff = get_df_return(df_mid)

In [29]:
lags=10

df=df_mid.copy()
for px in pxlst: 
    df[f'Forward_Return_{px}'] = np.log(df[px].shift(-1) / df[px])
for i in range(1, lags+1):
    for px in pxlst:
        df[f'{px}_Lag_{i}'] = df[f'Forward_Return_{px}'].shift(i)
df.dropna(inplace=True)
mean = df_mid.mean()
std = df_mid.std()
for px in pxlst: 
    df[f'Dev_{px}'] = df[px] - mean[px]

lst_max_lags=[]
lst_coef = []
lst_max_r2 = []
for px in pxlst:
    max_r2=-1e9
    for now_lag in range(1, lags+1):
        features = []
        #for px in pxlst:
        #    features.append( f'Dev_{px}' )
        for i in range(1, now_lag+1):
            for j in pxlst:
                features.append( f'{j}_Lag_{i}' )
        target = f'Forward_Return_{px}'
        
        X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.5, random_state=42)
        model = LinearRegression(fit_intercept=False)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        if max_r2<r2:
            max_r2=r2
            max_lags=now_lag
            max_coefficients = model.coef_
    lst_max_r2.append(max_r2)
    lst_max_lags.append(max_lags)
    lst_coef.append(list(max_coefficients))

print(lst_max_r2)
print(lst_max_lags)
print(lst_coef)

[0.33826443382841964, 0.33649781564653647, 0.07279168524449664, 0.043894598360596504]
[10, 10, 10, 10]
[[-0.7188483239556117, 0.010901776085869563, -0.0006328359762789832, 0.000561065044186296, -0.4497651634453357, 0.02213100464555035, -0.005283894532568928, 0.004834173590827603, -0.23526185885628692, 0.032221156171144474, -0.0031303743292175413, 0.0012404854648495334, -0.06458375700179084, 0.04610782471900584, 0.004675635931071243, -0.003596873454856139, 0.06812297794566947, 0.072995846704406, -0.0010831311191177992, -0.002980180145479264, 0.16135038210451458, 0.07787167102219117, -0.007906079636590607, -0.007311282074882775, 0.20943198804618063, 0.08904326601735198, 0.000838012593953217, 0.0009143480505533873, 0.23361190470556373, 0.0834509373880746, 0.0015165949251101023, 0.002177812813281126, 0.20616651178319023, 0.06663521182104347, 0.0028781341497414984, 7.161151399256871e-05, 0.12925341644359017, 0.03895963042162065, 0.0003644972431904905, 0.0008455646024733395], [-0.02248964641

In [22]:
lags=10
hor=5

df=df_mid.copy()
for px in pxlst: 
    df[f'Forward_Return_{px}'] = np.log(df[px].shift(-1) / df[px])
    df[f'Forward_Return_{px}_{hor}'] = df[f'Forward_Return_{px}'].rolling(window=hor).sum().shift(1-hor)
for i in range(1, lags+1):
    for px in pxlst:
        df[f'{px}_Lag_{i}'] = df[f'Forward_Return_{px}'].shift(i)
df.dropna(inplace=True)

lst_max_lags=[]
lst_coef = []
lst_max_r2 = []
for px in pxlst:
    max_r2=-1e9
    for now_lag in range(1, lags+1):
        features = []
        for i in range(1, now_lag+1):
            for j in pxlst:
                features.append( f'{j}_Lag_{i}' )
        target = f'Forward_Return_{px}_{hor}'
        
        X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.5, random_state=42)
        model = LinearRegression(fit_intercept=False)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        if max_r2<r2:
            max_r2=r2
            max_lags=now_lag
            max_coefficients = model.coef_
    lst_max_r2.append(max_r2)
    lst_max_lags.append(max_lags)
    lst_coef.append(list(max_coefficients))

print(lst_max_r2)
print(lst_max_lags)
print(lst_coef)

[0.35798432231989663, 0.3519124489668658, 0.08528939959475812, 0.048757141765686685]
[10, 10, 10, 10]
[[-0.5823063360920407, 0.06358473814238058, -0.002352940532019413, 0.001045412529136158, -0.22169633926565596, 0.12093153381408468, -0.006466063527505497, -0.0048112944081081235, 0.07532775854842144, 0.16903750931604486, -0.005376539578941197, -0.0019341064696594397, 0.30969201421382847, 0.20696125053229392, -0.0022955716496897947, 0.004891664122625334, 0.47351864366713986, 0.22693145940644702, -0.003666281716434927, 0.0017718749095912178, 0.5617038186508428, 0.24471501559525063, -0.005038212910917089, -0.000217878718088367, 0.58297010928792, 0.23626201271851166, 0.0011095861578532822, 0.0006364319217189772, 0.5413574374141419, 0.21394933523116896, -0.0002885948209551855, 0.003639622799216335, 0.4261499097059581, 0.16428036871153429, -0.0009019527738283741, 0.0031952141010928013, 0.25020838534452317, 0.09338081527709227, 0.004351980316854528, 0.000605624811986428], [-0.0517897322919372